In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, GRU, Bidirectional, Dense, RepeatVector, TimeDistributed


import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('TurinAHU.csv')
df

In [ ]:
############################################################# STATEFUL LSTM #############################################################
# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

# Prepare the data for training
X = []
y = []
n_future = 2 # number of timesteps to predict
n_past = 96 # number of timesteps to use as input
batch_size = 64
for i in range(n_past, len(data_scaled) - n_future + 1):
    X.append(data_scaled[i - n_past:i, :])
    y.append(data_scaled[i:i + n_future, :])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
split_index = int(len(X) * 0.7)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Define the model architecture
model = Sequential()
model.add(LSTM(64, batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]), activation='tanh', stateful=True, return_sequences=True))
model.add(LSTM(64, activation='tanh', dropout=0.5, stateful=True, return_sequences=True))
model.add(LSTM(64, activation='tanh', dropout=0.5, stateful=True))
model.add(RepeatVector(n_future))
model.add(LSTM(64, activation='tanh', dropout=0.5, stateful=True, return_sequences=True))
model.add(LSTM(64, activation='tanh', dropout=0.5, stateful=True, return_sequences=True))
model.add(TimeDistributed(Dense(X_train.shape[2], activation='linear')))
model.compile(optimizer='adam', loss='mse')

# Train the model
epochs = 500
for i in range(epochs):
    print('Epoch', i+1, '/', epochs)
    history = model.fit(X_train, y_train, epochs=1, batch_size=batch_size, validation_split=0.2, shuffle=False)
    model.reset_states()

# Evaluate the model
y_pred = model.predict(X_test, batch_size=batch_size)
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2]))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(y_test.shape[0]*y_test.shape[1], y_test.shape[2]))

mae = np.mean(np.abs(y_pred_rescaled - y_test_rescaled), axis=0)
mape = np.mean(np.abs((y_pred_rescaled - y_test_rescaled) / y_test_rescaled), axis=0) * 100
rmse = np.sqrt(np.mean(np.square(y_pred_rescaled - y_test_rescaled), axis=0))

print('MAE:', mae)
print('MAPE:', mape)
print('RMSE:', rmse)